## Square GNN Experimentation

In [ ]:
NUMAGENTS = 30
NUMEXPERIMENTS = 100
TIME = 25 # seconds
VISUAL = False

In [ ]:
import simulation.Visualizer
import simulation.World
import simulation.Agent
import pickle
from matplotlib import pyplot as plt

simulation.Agent.USE = "SIM"
simulation.Agent.CASE = "SQU"

for i in range(NUMEXPERIMENTS):
    if i%10== 0:
        print(str(i)+"/"+str(NUMEXPERIMENTS))
    filename = "./data/squ-logs/experiment-"+str(i)+"-log.p"
    with open(filename, "wb") as f:
        world = simulation.World.World(f)
        for j in range(int(NUMAGENTS)):
            a = simulation.Agent.Agent(swarm=j%2)
            world.agents.append(a)

        if VISUAL:
            v = simulation.Visualizer.Visualizer(world)
            v.run()
        else:
            while world.stepCount < TIME*10:
                world.step()
            world.closeWorld()

In [ ]:
import shutil
from utils.SquDataset import SquDataset

###############################
###  Gather and split data  ###
###############################

ta = 70 # Percentage used for training
te = 15 # Percentage used for testing (remaining used for validation)

try:
    shutil.rmtree("/tmp/Squares", ignore_errors=False, onerror=None)
except FileNotFoundError as e:
    pass
    
dataset = SquDataset(root='/tmp/Squares').shuffle()
maxSize = 5000
dataset = dataset[:min(len(dataset), maxSize)]

train_data = dataset[0:int(ta/100.*len(dataset))]
test_data = dataset[int(ta/100.*len(dataset)):int((ta+te)/100.*len(dataset))]
validate_data = dataset[int((ta+te)/100.*len(dataset)):]

In [ ]:
from torch_geometric.data import Data, DataLoader
from utils.models import *
from tqdm import tqdm
import numpy as np
import torch
from time import sleep

np.random.seed(1337)

num_epochs = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainLoader = DataLoader(train_data, batch_size=32, shuffle=True)
testLoader = DataLoader(test_data, batch_size=32, shuffle=True)

node_dim = train_data[0].x.shape[1]
edge_dim = 0
if train_data[0].edge_attr is not None: # if edge attributes used
    edge_dim = train_data[0].edge_attr.shape[1]
out_dim = train_data[0].y.shape[1]

mod = GenericGNN(node_dim, edge_dim, out_dim)
optimizer = torch.optim.Adam(mod.parameters(), lr=4e-5, weight_decay=5e-7)

In [ ]:
from random import random
from copy import deepcopy as copy
from tqdm import tqdm
from time import sleep
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

plt.rcParams['figure.figsize'] = (12, 4)

lossFunction = nn.MSELoss()

def train(model, optimizer, dataset):
    
    model.to(device)
    model.train()
    total_loss = 0.0
    num_items = 1
        
    for row in dataset:
        optimizer.zero_grad()
        pred = model(row.x, 
                     row.edge_index, 
                     row.edge_attr)
        loss = lossFunction(row.y, pred)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_items += 1
    return total_loss/num_items

def evaluate(model, dataset):
    
    model.to(device)
    model.eval()
    
    total_loss = 0.0
    num_items = 0
    
    num_outs = dataset[0].y.shape[1]
    
    predHist = [[] for _ in range(num_outs)]
    actualHist = [[] for _ in range(num_outs)]    

    for row in dataset:
        pred = model(row.x, 
                 row.edge_index, 
                 row.edge_attr)
        loss = lossFunction(row.y, pred)

        for i in range(len(pred[0])):
            predHist[i].append(pred[0][i].item())
            actualHist[i].append(row.y[0][i].item())
            
        total_loss += loss.item()
        num_items += 1
        
    return total_loss/num_items, actualHist, predHist

In [ ]:
num_epochs = 30
epoch = 0
nbins  = 100
minmax = 0.5
bins  = [((i*(minmax*2))/nbins) - minmax for i in range(nbins+1)]
losses = []

plt.rcParams['figure.figsize'] = [5, 3]
plt.rcParams['figure.dpi'] = 100

for epoch in tqdm(range(epoch, num_epochs)):
       
    train_loss      = train(   mod, optimizer, train_data)
    test_loss, a, p = evaluate(mod,            test_data)
    
    losses.append([train_loss, test_loss])
    
    print("{train: " + str(train_loss) +"}, {test: " + str(test_loss) + "}")
    
    plt.rcParams['figure.figsize'] = (5, 3)    
    fig, ax = plt.subplots(1,1)
    fig.suptitle("Testing dataset", fontsize=16)
    ax.hist(p[0], bins=bins, alpha=0.5, label="predicted")
    ax.hist(a[0], bins=bins, alpha=0.5, label="actual")
    ax.legend()
    plt.show()
    
    torch.save(mod.state_dict(), './models/squ-models/SQU-'+str(epoch)+'.mod')
    sleep(0.5)
    
torch.save(mod.state_dict(), './models/squ-models/SQU-FINAL.mod')

In [ ]:
plt.plot([i for i in range(len(losses))],[losses[i][0] for i in range(len(losses))])
plt.plot([i for i in range(len(losses))],[losses[i][1] for i in range(len(losses))])

In [ ]:
mod = GenericGNN(node_dim, edge_dim, out_dim)
mod.load_state_dict(torch.load('./models/squ-models/SQU-1.mod'))
mod.eval()

In [ ]:
datapoints = 10000
from random import random
import math

dataFileName = 'randomData.csv'
dataString = ''

MULT = 1
TARGET_DIST = 0.15 
HYP = math.sqrt(pow(TARGET_DIST,2)+pow(TARGET_DIST,2))
DIP = 1 


def lj(x, e, t=None):
    
    if t is None:
        t = TARGET_DIST
    
    dist = math.sqrt(((x[1][0]-x[0][0])**2)+((x[1][1]-x[0][1])**2))
    epsilon = DIP*MULT
    sigma = (t*MULT)/(2**(1/6))
    mag = min((epsilon*4) * (pow((sigma/dist),12)-pow((sigma/dist),6)), 10)
    
    angle = 0
    if mag < 0:
        angle = math.pi
    
    return mag, angle

minDist = 0.1
maxDist = 0.4

mod.eval()

mod.to('cpu')

scatterX = []
scatterY = []
scatterA = []
scatterY2 = []
scatterA2 = []
scatterAA = []
scatterAY = []
scatterAA2 = []
scatterAY2 = []


ljX = []
ljY = []
ljY2 = []
for i in range(1000):
    delta = (maxDist-minDist)/1000
    ljX.append(minDist + (i*delta))
    m,a = lj([[0, 0], [ljX[-1], 0]], [])
    m2,a2 = lj([[0, 0], [ljX[-1], 0]], [], HYP)
    ljY.append(m)
    ljY2.append(m2)

for i in range(datapoints):
    
    data = [[0, 0] for _ in range(7)]
    
    ox = random()
    oy = random()
    d = (maxDist-minDist) * random() + minDist
    scatterX.append(d)
    a = random() * (2*math.pi)
    no = ox + d*math.cos(a)
    ny = oy + d*math.sin(a)
    
    data[0] = [ox, oy]
    data[1] = [no, ny]    
    
    e1 = [0, 1]
    e2 = [1, 0]
    
    x = torch.tensor(data, dtype=torch.float)
    e = torch.tensor([e1, e2], dtype=torch.long)
    a1 = torch.tensor([[1], [1]], dtype=torch.float)
    a2 = torch.tensor([[2], [2]], dtype=torch.float)
    pred1 = mod(x, e, a1)
    pred2 = mod(x, e, a2)
    ljM, ljA = lj(x, e)
    ljM2, ljA2 = lj(x, e, HYP)
    
    mag = math.sqrt(pred1[0][0]**2 + pred1[0][1]**2)
    ang = math.atan2(pred1[0][1], pred1[0][0]) - math.atan2(oy-ny, ox-no)
    mag2 = math.sqrt(pred2[0][0]**2 + pred2[0][1]**2)
    ang2 = math.atan2(pred2[0][1], pred2[0][0]) - math.atan2(oy-ny, ox-no)
    
    if ang < 0:
        ang = -ang
    if ang > math.pi:
        ang = ang-math.pi
        ang = -ang
        ang = ang+math.pi
    if ang > math.pi/2:
        mag = -mag
        
    if ang2 < 0:
        ang2 = -ang2
    if ang2 > math.pi:
        ang2 = ang2-math.pi
        ang2 = -ang2
        ang2 = ang2+math.pi
    if ang2 > math.pi/2:
        mag2 = -mag2
        
    scatterY.append(mag)
    scatterA.append(ang)
    scatterY2.append(mag2)
    scatterA2.append(ang2)
    scatterAA.append(ljA)
    scatterAY.append(ljM)
    scatterAA2.append(ljA2)
    scatterAY2.append(ljM2)
    
plt.rcParams['figure.figsize'] = [5, 4]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

ys= -1.2
ye = 1

plt.title('Magnitude of Acceleration for Square Lattice')
plt.plot([0.5, 0.5], [ys, ye], 'k--')
plt.xlabel("Distance Between Two Agents")
plt.ylabel("Magnitude of Acceleration")
plt.xlim([minDist, maxDist])
plt.ylim([ys, ye])
plt.scatter(scatterX, scatterY, s=1, alpha=0.2, label='Neighbors k=1')
plt.scatter(scatterX, scatterY2, s=1, alpha=0.2, label='Neighbors k=2')
plt.plot(ljX, ljY, 'r--', label='Actual LJ Potentials')
plt.plot(ljX, ljY2, 'r--')
plt.plot([minDist, maxDist], [0, 0], 'k--')
plt.legend()

s1 = ""
s2 = ""
for i in range(len(scatterX)):
    s1 += str(scatterX[i])+","+str(scatterY[i]) + "\n"
    s2 += str(scatterX[i])+","+str(scatterY2[i]) + "\n"

with open("./data/MME-data/SQUAREDATA1.csv", "w") as f:
    f.write(s1)
with open("./data/MME-data/SQUAREDATA2.csv", "w") as f:
    f.write(s2)   
